In [ ]:
from google.colab import drive
drive.mount('/content/drive')

address = "Colab Notebooks"
import sys
sys.path.append('/content/drive/My Drive/{}'.format(address))

%cd /content/drive/My\ Drive/$address

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

# Add Reranker

In [ ]:
import pickle
with open('./vectorstore.pkl', 'rb') as f:
    vectorstore = pickle.load(f)

In [ ]:
query = "A man tells his life story to strangers who sit next to him on a bench at a bus stop"
docs = vectorstore.similarity_search(query, k=100)

In [ ]:
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast

model_path = "movie-genre-detection-test1"
classification_model = BertForSequenceClassification.from_pretrained(model_path)
classification_tokenizer= BertTokenizerFast.from_pretrained(model_path)
nlp = pipeline("sentiment-analysis", model=classification_model, tokenizer=classification_tokenizer)

In [ ]:
def extract_document_genre(doc):
  for line in doc.page_content.split('\n'):
    if not 'Genre' in line:
      continue
    return line.split(':')[1].strip()


def filter_by_genre(query, documents):
  base_genre = nlp(query)[0]['label']
  valid_docs = []
  for document in documents:
    doc_genre = extract_document_genre(document)
    if base_genre.lower() in doc_genre.lower():
      valid_docs.append(document)
  return valid_docs

In [ ]:
valid_docs = filter_by_genre(query, docs)
len(valid_docs), len(docs)

(46, 100)

In [ ]:
from sentence_transformers import CrossEncoder

model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2", max_length=512)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
def rerank_results(query, documents, top_n):
  model_inputs = [[query, doc.page_content] for doc in documents]
  scores = model.predict(model_inputs)
  results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
  final_results = []
  for r in results[:top_n]:
    doc = documents[r[0]]
    doc.metadata["relevance_score"] = r[1]
    final_results.append(doc)
  return final_results

In [ ]:
docs = filter_by_genre(query, docs)
reranked_list = rerank_results(query, docs, 5)
for d in reranked_list:
  print(d.page_content)

: 17503
Genre: drama
Origin/Ethnicity: Australian
Plot: Jack, a young British hitchhiker in France, attempts to get a ride to the ferry service back to England. After walking along the road for hours without having any cars stop, he comes upon an arguing couple who pull over and continue their fight. Concerned, Jack approaches the car and pulls a man out before he can strike a woman. The man backs off from Jack and drives away, leaving Jack with the woman, who introduces herself as a fellow hitchhiker named Véronique. Véronique tells him that there is a serial killer active on this road, which explains why Jack could not get a ride. The two agree to travel together for safety, and Véronique helps him work on his French. Véronique says she is a drifter whose parents blame her for her brother's accidental death, and Jack that, on the same day, he lost all his luggage and found his girlfriend cheating on him.
An eccentric man, Grizard, stops and offers the two a ride. Although Véronique i